## Imports

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import matplotlib.pyplot as plt

## Data

In [11]:
df = pd.read_csv('icl_train.zip', 
                 compression='zip',
                 sep=',',
                 parse_dates=['date'],
                 index_col='date')
df.head()

,zone,Вызов специальной бригады,Перевозка плановая,Перевозка экстренная,авария ( сбило машиной) постр.,аллергия( задыхается),аллергия( отекло лицо),аллергия( сыпь),аритмия,без сознания,...,травма позвоночника,травма ребер( задыхается),травма ребер( не задыхается),травма руки,травма руки( кровотечение),тяж.ст.опьянения,укусила собака,умер?,умер?( хронический больной),умирает
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01 07:00:00,П/станция 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-01-01 08:00:00,П/станция 1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2019-01-01 09:00:00,П/станция 1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2019-01-01 10:00:00,П/станция 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2019-01-01 11:00:00,П/станция 1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Погода

In [23]:
weather_df = pd.read_csv('weather_prepared_interpolated.csv', 
                         encoding='cp1251',
                         parse_dates=[0],
                         index_col=0)
weather_df.head()

,weather,temp,w_spid,p
2019-01-01 03:00:00,пасмурно,-8.000000,3.000000,765.0
2019-01-01 04:00:00,пасмурно,-8.333333,2.666667,765.0
2019-01-01 05:00:00,пасмурно,-8.666667,2.333333,765.0
2019-01-01 06:00:00,пасмурно,-9.000000,2.000000,765.0
2019-01-01 07:00:00,пасмурно,-9.333333,2.000000,765.0


#### Лунные фазы

In [24]:
moon_df = pd.read_csv('moon_prepared_interpolated.csv', 
                      encoding='cp1251',
                      parse_dates=[0],
                      index_col=0)
moon_df.head()

,новая луна,первая четверь,полнолуние,третья четверть
2019-01-01 00:00:00,1.0,0.0,0.0,0.0
2019-01-01 01:00:00,1.0,0.0,0.0,0.0
2019-01-01 02:00:00,1.0,0.0,0.0,0.0
2019-01-01 03:00:00,1.0,0.0,0.0,0.0
2019-01-01 04:00:00,1.0,0.0,0.0,0.0


#### Динамика по ковиду

In [43]:
covid_df = pd.read_csv('covid_interpolated.csv', 
                       encoding='cp1251',
                       parse_dates=[0],
                       index_col=0)
covid_df.head()

,Заражений за день
2019-01-01 00:00:00,0.0
2019-01-01 01:00:00,0.0
2019-01-01 02:00:00,0.0
2019-01-01 03:00:00,0.0
2019-01-01 04:00:00,0.0


### Агрегируем данные

In [44]:
df_new = df.copy()
df_new[weather_df.columns] = weather_df.loc[df.index]
df_new[moon_df.columns] = moon_df.loc[df.index]
df_new[covid_df.columns] = covid_df.loc[df.index]

In [45]:
df_new.head()

,zone,Вызов специальной бригады,Перевозка плановая,Перевозка экстренная,авария ( сбило машиной) постр.,аллергия( задыхается),аллергия( отекло лицо),аллергия( сыпь),аритмия,без сознания,...,умирает,weather,temp,w_spid,p,новая луна,первая четверь,полнолуние,третья четверть,Заражений за день
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01 07:00:00,П/станция 1,0,0,0,0,0,0,0,0,0,...,0,пасмурно,-9.333333,2.000000,765.000000,1.0,0.0,0.0,0.0,0.0
2019-01-01 08:00:00,П/станция 1,0,0,0,0,0,0,0,0,0,...,1,пасмурно,-9.666667,2.000000,765.000000,1.0,0.0,0.0,0.0,0.0
2019-01-01 09:00:00,П/станция 1,0,0,0,0,0,0,0,0,1,...,1,малооблачно,-10.000000,2.000000,765.000000,1.0,0.0,0.0,0.0,0.0
2019-01-01 10:00:00,П/станция 1,0,0,0,0,0,0,0,0,0,...,0,малооблачно,-9.666667,2.006102,764.666667,1.0,0.0,0.0,0.0,0.0
2019-01-01 11:00:00,П/станция 1,1,0,0,0,0,0,0,0,0,...,0,малооблачно,-9.333333,2.012205,764.333333,1.0,0.0,0.0,0.0,0.0


## Pipeline